# C-More

In [1]:
import tweepy
import json
import time
import pandas as pd

from nltk.sentiment.vader import SentimentIntensityAnalyzer
from textblob import TextBlob
import textacy.resources

In [2]:
# Twitter API v2

bearer_token = ''

client = tweepy.Client(bearer_token=bearer_token)

### 1. Telco (Vodafone)

#### 1.1. Get tweets

In [3]:
query = "Vodafone 5G -is:retweet lang:en"

# maches tweets with words Vodafone and 5G, excluding retweets, for tweets in EN (app language)

In [4]:
response = client.get_recent_tweets_count(query, granularity="day") # count of tweets for the last 7 days
# granularity can be minute, hour or day

In [5]:
for count in response.data:
    print(count)

{'end': '2022-07-13T00:00:00.000Z', 'start': '2022-07-12T17:24:15.000Z', 'tweet_count': 12}
{'end': '2022-07-14T00:00:00.000Z', 'start': '2022-07-13T00:00:00.000Z', 'tweet_count': 65}
{'end': '2022-07-15T00:00:00.000Z', 'start': '2022-07-14T00:00:00.000Z', 'tweet_count': 28}
{'end': '2022-07-16T00:00:00.000Z', 'start': '2022-07-15T00:00:00.000Z', 'tweet_count': 29}
{'end': '2022-07-17T00:00:00.000Z', 'start': '2022-07-16T00:00:00.000Z', 'tweet_count': 16}
{'end': '2022-07-18T00:00:00.000Z', 'start': '2022-07-17T00:00:00.000Z', 'tweet_count': 21}
{'end': '2022-07-19T00:00:00.000Z', 'start': '2022-07-18T00:00:00.000Z', 'tweet_count': 65}
{'end': '2022-07-19T17:24:15.000Z', 'start': '2022-07-19T00:00:00.000Z', 'tweet_count': 41}


In [6]:
response = client.search_recent_tweets(query, 
                                       end_time="2022-07-18T00:00:00Z", 
                                       max_results=100, 
                                       tweet_fields=['lang', 'created_at', 'public_metrics'])

In [7]:
for tweet in response.data:
    print(tweet.id, tweet.text, tweet.lang, tweet.created_at)

1548818090401714177 Towns and smaller cities set to lose out on 5G | Vodafone report - Vodafone UK News Centre #5G  CC:@UK_CIF https://t.co/OiBvjPV35T en 2022-07-17 23:53:03+00:00
1548817148147032068 Forbes deep-dives into how Vodafone is revamping its operations with @Oracle to reduce costs and deliver the next generation of #5G and #IoT services. https://t.co/ozIpyLeXRx https://t.co/r1qa1GAt68 en 2022-07-17 23:49:18+00:00
1548815541837500416 @SamsungAU Any reason 4g and 5g is almost un usable on @VodafoneAU using S22 Ultra. Vodafone are saying it's a Samsung problem? en 2022-07-17 23:42:55+00:00
1548796630748602368 Forbes deep-dives into how Vodafone is revamping its operations with @Oracle to reduce costs and deliver the next generation of #5G and #IoT services. https://t.co/Rp6wyB9Juk https://t.co/OlFgzoxqbm en 2022-07-17 22:27:46+00:00
1548760366829690887 #5gradiation #swindon #northswindon #wiltshire #youth #fearless #swindondeservesbetter #Vodafone #EMF2022 #emf #colchester #knu

#### 1.2. Store tweets in data frame

In [8]:
data = [[tweet.id, 
         tweet.text,
         tweet.lang,
         tweet.public_metrics['retweet_count'], 
         tweet.public_metrics['reply_count'], 
         tweet.public_metrics['like_count'], 
         tweet.public_metrics['quote_count'], 
         tweet.created_at
        ] for tweet in response.data]

df_vodafone = pd.DataFrame(data, columns = ['id', 'text', 'lang', 'retweets', 'replies', 'likes', 'quotes', 'created_at'])

In [9]:
df_vodafone.head()

,id,text,lang,retweets,replies,likes,quotes,created_at
0,1548818090401714177,Towns and smaller cities set to lose out on 5G...,en,0,0,0,0,2022-07-17 23:53:03+00:00
1,1548817148147032068,Forbes deep-dives into how Vodafone is revampi...,en,0,0,0,0,2022-07-17 23:49:18+00:00
2,1548815541837500416,@SamsungAU Any reason 4g and 5g is almost un u...,en,0,1,0,0,2022-07-17 23:42:55+00:00
3,1548796630748602368,Forbes deep-dives into how Vodafone is revampi...,en,0,0,0,0,2022-07-17 22:27:46+00:00
4,1548760366829690887,#5gradiation #swindon #northswindon #wiltshire...,en,2,0,1,0,2022-07-17 20:03:40+00:00


In [10]:
df_vodafone.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype              
---  ------      --------------  -----              
 0   id          100 non-null    int64              
 1   text        100 non-null    object             
 2   lang        100 non-null    object             
 3   retweets    100 non-null    int64              
 4   replies     100 non-null    int64              
 5   likes       100 non-null    int64              
 6   quotes      100 non-null    int64              
 7   created_at  100 non-null    datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](1), int64(5), object(2)
memory usage: 6.4+ KB


In [11]:
df_vodafone['lang'].value_counts()

en    100
Name: lang, dtype: int64

All our tweets are in English, so we can drop this column.

In [12]:
df_vodafone.drop('lang', axis=1, inplace=True)

In [13]:
df_vodafone.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype              
---  ------      --------------  -----              
 0   id          100 non-null    int64              
 1   text        100 non-null    object             
 2   retweets    100 non-null    int64              
 3   replies     100 non-null    int64              
 4   likes       100 non-null    int64              
 5   quotes      100 non-null    int64              
 6   created_at  100 non-null    datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](1), int64(5), object(1)
memory usage: 5.6+ KB


#### 1.3. Sentiment Analysis with VADER

In [14]:
sid = SentimentIntensityAnalyzer()

In [15]:
df_vodafone['scores'] = df_vodafone['text'].map(lambda tweet: sid.polarity_scores(tweet))

df_vodafone['vader_score']  = df_vodafone['scores'].map(lambda score_dict: score_dict['compound'])

df_vodafone['vader_label'] = df_vodafone['vader_score'].map(lambda comp: 'pos' if comp >=0.05 else ('neg' if comp<=-0.05 else 'neu'))

In [51]:
df_vodafone

,id,text,retweets,replies,likes,quotes,created_at,scores,vader_score,vader_label
0,1548818090401714177,Towns and smaller cities set to lose out on 5G...,0,0,0,0,2022-07-17 23:53:03+00:00,"{'neg': 0.13, 'neu': 0.87, 'pos': 0.0, 'compou...",-0.4019,neg
1,1548817148147032068,Forbes deep-dives into how Vodafone is revampi...,0,0,0,0,2022-07-17 23:49:18+00:00,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,neu
2,1548815541837500416,@SamsungAU Any reason 4g and 5g is almost un u...,0,1,0,0,2022-07-17 23:42:55+00:00,"{'neg': 0.119, 'neu': 0.881, 'pos': 0.0, 'comp...",-0.4019,neg
3,1548796630748602368,Forbes deep-dives into how Vodafone is revampi...,0,0,0,0,2022-07-17 22:27:46+00:00,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,neu
4,1548760366829690887,#5gradiation #swindon #northswindon #wiltshire...,2,0,1,0,2022-07-17 20:03:40+00:00,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,neu
...,...,...,...,...,...,...,...,...,...,...
95,1547273112328146945,"The Adani group, through unit Adani Data Netwo...",0,0,0,0,2022-07-13 17:33:51+00:00,"{'neg': 0.0, 'neu': 0.858, 'pos': 0.142, 'comp...",0.5106,pos
96,1547258380950990848,Vodafone and VMO2 will deploy 4G and 5G on Lon...,0,0,0,0,2022-07-13 16:35:19+00:00,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,neu
97,1547252964360097792,Adani’s entry in 5G may heat up bidding race\n...,0,1,1,0,2022-07-13 16:13:47+00:00,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,neu
98,1547252953337442305,Adani group to participate in 5G spectrum auct...,0,1,1,0,2022-07-13 16:13:45+00:00,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,neu


In [16]:
df_vodafone['vader_label'].value_counts()

neu    47
pos    44
neg     9
Name: vader_label, dtype: int64

In [53]:
print(df_vodafone[df_vodafone['vader_label'] == 'pos']['text'].values)

['Amazing 5G throughout at super crowded location with Vodafone and Ericsson. @vodafone_de, @Ericsson_GmbH #5G #duesseldorf #sommerkirmis https://t.co/w7xzZGeb5M'
 'What an incredible initiative from @VodafoneUK. With the use of #5G and haptic suits, deaf and hard-of hearing fans will now feel every beat through vibration at festivals. https://t.co/Uw0oBO7FGo https://t.co/LhURU7PMz5'
 'The latest Telecom Pulse - Vignesh Iyer Daily! https://t.co/k4TC6ZWXkO Thanks to @NpoReports @hornbilltv #5g #vodafone'
 'You can now capture and share every moment of your adventure through Vodafone’s Data Roaming. Enjoy free additional roaming data and roaming minutes on all Unlimited 5G plans. Check out our 5G Unlimited plans and roam worry-free with Vodafone! T&amp;Cs apply.\nhttps://t.co/crqgRDxqqn https://t.co/Dv2p7tAYsV'
 'Google Cloud has arguably been at the forefront of telco/hyperscaler engagements in 2022, having also announced key partnerships with the likes of Vodafone, BT, Telefónica Deuts

In [17]:
print(df_vodafone[df_vodafone['vader_label'] == 'neg']['text'].values)

['Towns and smaller cities set to lose out on 5G | Vodafone report - Vodafone UK News Centre #5G  CC:@UK_CIF https://t.co/OiBvjPV35T'
 "@SamsungAU Any reason 4g and 5g is almost un usable on @VodafoneAU using S22 Ultra. Vodafone are saying it's a Samsung problem?"
 'Fake claim in circulation that people are dying due to 5G trials and not COVID-19 - Oneindia News\n\nhttps://t.co/VYK3KeawR5\n\n #AIRTEL #Jio #vodafone'
 "Thank You @ViCustomerCare!!\n#Vodafone #Vi #vodafoneidea #CustomerCare\nEvery mobile operator right now: We are bringing 5G\nVodafoneIdea : Why don't you see how 2G was?\nDon't even want to waste my time with customer service now. It's the 5th time I am raising this issue. https://t.co/Ic1VZgz1Pg"
 '@eatsruns Sadly not - I was with John Lewis before, and just switched to Vodafone, but not 5G.'
 "What if Bankruptcy of Vodafone-Idea followed by Bailout by govt of India and Adani's bid for 5G spectrum are related?\n\n#5Gauction"
 "What an useless Govt it is. Playing dirty ma

Nota: para a análise de sentimento, pode ser importante excluir as replies - o tweet original pode ter um determinado sentimento associado e uma dada resposta um sentimento oposto porque estamos a ignorar o contexto do tweet original.

Exemplo: um tweet em que alguém que se queixa de um problema com a vodafone (sentimento negativo) a que outra pessoa responde com uma solução referindo que a operadora X é muito melhor (sentimento positivo, mas que acaba por ser negativo para a Vodafone).

In [18]:
df_vodafone.drop('scores', axis=1, inplace=True)

In [19]:
df_vodafone.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype              
---  ------       --------------  -----              
 0   id           100 non-null    int64              
 1   text         100 non-null    object             
 2   retweets     100 non-null    int64              
 3   replies      100 non-null    int64              
 4   likes        100 non-null    int64              
 5   quotes       100 non-null    int64              
 6   created_at   100 non-null    datetime64[ns, UTC]
 7   vader_score  100 non-null    float64            
 8   vader_label  100 non-null    object             
dtypes: datetime64[ns, UTC](1), float64(1), int64(5), object(2)
memory usage: 7.2+ KB


#### 1.4. Sentiment Analysis with TextBlob

In [20]:
corpus = textacy.Corpus("en_core_web_sm", df_vodafone['text'])

In [21]:
print(corpus)

Corpus(100 docs, 3473 tokens)


In [22]:
pol = []

for doc in corpus:
    
    tblob = TextBlob(doc.text)
    
    pol.append({'polarity': tblob.sentiment.polarity})

In [23]:
df_vodafone['tblob_score'] = pd.DataFrame(pol)

In [24]:
df_vodafone.head()

,id,text,retweets,replies,likes,quotes,created_at,vader_score,vader_label,tblob_score
0,1548818090401714177,Towns and smaller cities set to lose out on 5G...,0,0,0,0,2022-07-17 23:53:03+00:00,-0.4019,neg,0.00
1,1548817148147032068,Forbes deep-dives into how Vodafone is revampi...,0,0,0,0,2022-07-17 23:49:18+00:00,0.0000,neu,0.00
2,1548815541837500416,@SamsungAU Any reason 4g and 5g is almost un u...,0,1,0,0,2022-07-17 23:42:55+00:00,-0.4019,neg,0.00
3,1548796630748602368,Forbes deep-dives into how Vodafone is revampi...,0,0,0,0,2022-07-17 22:27:46+00:00,0.0000,neu,0.00
4,1548760366829690887,#5gradiation #swindon #northswindon #wiltshire...,2,0,1,0,2022-07-17 20:03:40+00:00,0.0000,neu,0.25


In [25]:
df_vodafone['tblob_label'] = df_vodafone['tblob_score'].map(lambda pol: 'pos' if pol >=0.05 else ('neg' if pol <=-0.05 else 'neu'))

In [26]:
df_vodafone.head()

,id,text,retweets,replies,likes,quotes,created_at,vader_score,vader_label,tblob_score,tblob_label
0,1548818090401714177,Towns and smaller cities set to lose out on 5G...,0,0,0,0,2022-07-17 23:53:03+00:00,-0.4019,neg,0.00,neu
1,1548817148147032068,Forbes deep-dives into how Vodafone is revampi...,0,0,0,0,2022-07-17 23:49:18+00:00,0.0000,neu,0.00,neu
2,1548815541837500416,@SamsungAU Any reason 4g and 5g is almost un u...,0,1,0,0,2022-07-17 23:42:55+00:00,-0.4019,neg,0.00,neu
3,1548796630748602368,Forbes deep-dives into how Vodafone is revampi...,0,0,0,0,2022-07-17 22:27:46+00:00,0.0000,neu,0.00,neu
4,1548760366829690887,#5gradiation #swindon #northswindon #wiltshire...,2,0,1,0,2022-07-17 20:03:40+00:00,0.0000,neu,0.25,pos


In [27]:
df_vodafone['tblob_label'].value_counts()

neu    51
pos    41
neg     8
Name: tblob_label, dtype: int64

In [28]:
len(df_vodafone[(df_vodafone['vader_label'] == df_vodafone['tblob_label'])])

76

Os resultados do VADER e do TextBlob são concordantes em 76 dos 100 tweets.

#### 1.4. Emotion Analysis with DepecheMood

In [29]:
rs = textacy.resources.DepecheMood(lang="en", word_rep="lemma", min_freq=2)

moods = []

for doc in corpus:
    
    mood = sorted(rs.get_emotional_valence(doc).items(), key=lambda x: x[1], reverse=True)
    
    moods.append({'mood': mood})

In [30]:
df_vodafone['mood'] = pd.DataFrame(moods)

In [31]:
df_vodafone

,id,text,retweets,replies,likes,quotes,created_at,vader_score,vader_label,tblob_score,tblob_label,mood
0,1548818090401714177,Towns and smaller cities set to lose out on 5G...,0,0,0,0,2022-07-17 23:53:03+00:00,-0.4019,neg,0.000000,neu,"[(INSPIRED, 0.14742941371713533), (SAD, 0.1375..."
1,1548817148147032068,Forbes deep-dives into how Vodafone is revampi...,0,0,0,0,2022-07-17 23:49:18+00:00,0.0000,neu,0.000000,neu,"[(INSPIRED, 0.1535394550512804), (ANGRY, 0.133..."
2,1548815541837500416,@SamsungAU Any reason 4g and 5g is almost un u...,0,1,0,0,2022-07-17 23:42:55+00:00,-0.4019,neg,0.000000,neu,"[(ANGRY, 0.16900189733218576), (ANNOYED, 0.139..."
3,1548796630748602368,Forbes deep-dives into how Vodafone is revampi...,0,0,0,0,2022-07-17 22:27:46+00:00,0.0000,neu,0.000000,neu,"[(INSPIRED, 0.1535394550512804), (ANGRY, 0.133..."
4,1548760366829690887,#5gradiation #swindon #northswindon #wiltshire...,2,0,1,0,2022-07-17 20:03:40+00:00,0.0000,neu,0.250000,pos,"[(AMUSED, 0.1854112818637807), (INSPIRED, 0.17..."
...,...,...,...,...,...,...,...,...,...,...,...,...
95,1547273112328146945,"The Adani group, through unit Adani Data Netwo...",0,0,0,0,2022-07-13 17:33:51+00:00,0.5106,pos,0.150000,pos,"[(DONT_CARE, 0.18849327811381925), (AMUSED, 0...."
96,1547258380950990848,Vodafone and VMO2 will deploy 4G and 5G on Lon...,0,0,0,0,2022-07-13 16:35:19+00:00,0.0000,neu,0.000000,neu,"[(DONT_CARE, 0.15036639676231625), (INSPIRED, ..."
97,1547252964360097792,Adani’s entry in 5G may heat up bidding race\n...,0,1,1,0,2022-07-13 16:13:47+00:00,0.0000,neu,0.000000,neu,"[(AMUSED, 0.1547223483169548), (DONT_CARE, 0.1..."
98,1547252953337442305,Adani group to participate in 5G spectrum auct...,0,1,1,0,2022-07-13 16:13:45+00:00,0.0000,neu,0.125000,pos,"[(INSPIRED, 0.15171280796571587), (AMUSED, 0.1..."


In [32]:
df_vodafone['mood'].map(lambda x: len(x)).value_counts()

8    98
0     2
Name: mood, dtype: int64

It was not possible to analyse the emotion of 2 tweets.

In [33]:
df_vodafone['mood'][2]

[('ANGRY', 0.16900189733218576),
 ('ANNOYED', 0.13953618221387204),
 ('INSPIRED', 0.1393530094666046),
 ('AMUSED', 0.12893849653333483),
 ('DONT_CARE', 0.11759330102919252),
 ('HAPPY', 0.11060971045893975),
 ('SAD', 0.1068556063931119),
 ('AFRAID', 0.0881117965727586)]

In [34]:
df_vodafone['text'][2]

"@SamsungAU Any reason 4g and 5g is almost un usable on @VodafoneAU using S22 Ultra. Vodafone are saying it's a Samsung problem?"

The results make sense for this particular tweet.

In [35]:
df_vodafone['top1_mood'] = df_vodafone['mood'].map(lambda x: x[0][0] if len(x) > 0 else None)
df_vodafone['top2_mood'] = df_vodafone['mood'].map(lambda x: x[1][0] if len(x) > 0 else None)
df_vodafone['top3_mood'] = df_vodafone['mood'].map(lambda x: x[2][0] if len(x) > 0 else None)

In [36]:
df_vodafone.drop(['id', 'mood'], axis=1, inplace=True)

In [37]:
df_vodafone

,text,retweets,replies,likes,quotes,created_at,vader_score,vader_label,tblob_score,tblob_label,top1_mood,top2_mood,top3_mood
0,Towns and smaller cities set to lose out on 5G...,0,0,0,0,2022-07-17 23:53:03+00:00,-0.4019,neg,0.000000,neu,INSPIRED,SAD,ANGRY
1,Forbes deep-dives into how Vodafone is revampi...,0,0,0,0,2022-07-17 23:49:18+00:00,0.0000,neu,0.000000,neu,INSPIRED,ANGRY,ANNOYED
2,@SamsungAU Any reason 4g and 5g is almost un u...,0,1,0,0,2022-07-17 23:42:55+00:00,-0.4019,neg,0.000000,neu,ANGRY,ANNOYED,INSPIRED
3,Forbes deep-dives into how Vodafone is revampi...,0,0,0,0,2022-07-17 22:27:46+00:00,0.0000,neu,0.000000,neu,INSPIRED,ANGRY,ANNOYED
4,#5gradiation #swindon #northswindon #wiltshire...,2,0,1,0,2022-07-17 20:03:40+00:00,0.0000,neu,0.250000,pos,AMUSED,INSPIRED,DONT_CARE
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,"The Adani group, through unit Adani Data Netwo...",0,0,0,0,2022-07-13 17:33:51+00:00,0.5106,pos,0.150000,pos,DONT_CARE,AMUSED,ANNOYED
96,Vodafone and VMO2 will deploy 4G and 5G on Lon...,0,0,0,0,2022-07-13 16:35:19+00:00,0.0000,neu,0.000000,neu,DONT_CARE,INSPIRED,AFRAID
97,Adani’s entry in 5G may heat up bidding race\n...,0,1,1,0,2022-07-13 16:13:47+00:00,0.0000,neu,0.000000,neu,AMUSED,DONT_CARE,INSPIRED
98,Adani group to participate in 5G spectrum auct...,0,1,1,0,2022-07-13 16:13:45+00:00,0.0000,neu,0.125000,pos,INSPIRED,AMUSED,ANNOYED


In [38]:
df_vodafone.to_csv('vodafone.csv', index=False)